In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score

from imblearn.over_sampling import SMOTE

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.functions import load_OU_data, CourseScaler, plot_confusion, score_grid, \
smotecourses, process_courses, course_cross_validate, Course_GridSearchCV, \
model_evaluate_presentation, cross_val_presentation, GridSearchPresentationCV, \
graph_model_history

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

import keras
import tensorflow as tf
from keras.layers import Dense, Dropout
from keras.models import Sequential, load_model
from keras import optimizers, layers
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint

import warnings

from IPython.display import display

warnings.filterwarnings("ignore")
sns.set_style('white')
pd.set_option('display.max_columns',None)
tf.keras.backend.set_floatx('float64')

In [ ]:
df_full = load_OU_data(prediction_window=.5)

df = df_full.drop(columns = ['id_student','region','highest_education', \
                   'imd_band','gender','age_band','disability','studied_credits',
                   'module_presentation_length','date_registration', 'num_of_prev_attempts'])

df['final_result'] = np.array([1 if w in ['Pass','Distinction'] \
              else 0 for w in df['final_result']])

test_df = df[df['code_presentation'] == '2014J']

train_df = df[df['code_presentation'] != '2014J']

X_train = train_df.drop(columns = ['final_result','code_presentation'])
y_train = train_df['final_result']
X_test = test_df.drop(columns = ['final_result','code_presentation'])
y_test = test_df['final_result']


X_train_transformed, y_train_transformed, X_test_transformed \
                                = process_courses(X_train, y_train, X_test)



In [ ]:
def make_model1():
    model = Sequential()
    model.add(Dense(5, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

    return model
mc = ModelCheckpoint(filepath='../models/densemodel1.h5', save_best_only=True, monitor='val_acc',
                    mode = 'max')
model = make_model1()
history = model.fit(X_train_transformed, y_train_transformed,
                  epochs = 100,
                  validation_data = (X_test_transformed, y_test),
                   batch_size = 500,
                   verbose = 0,
                   callbacks = [mc])
model = load_model('../models/densemodel1.h5')
y_pred = np.round(model.predict(X_test_transformed))
graph_model_history(history)
print(accuracy_score(y_test, y_pred))
plot_confusion(y_test, y_pred)


In [ ]:
def make_model2():
    model = Sequential()
    model.add(Dense(5, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(5, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

    return model
mc = ModelCheckpoint(filepath='../models/densemodel2.h5', save_best_only=True, monitor='val_acc',
                    mode = 'max')
model = make_model2()
history = model.fit(X_train_transformed, y_train_transformed,
                  epochs = 100,
                  validation_data = (X_test_transformed,y_test),
                   batch_size = 500,
                   verbose = 0,
                   callbacks = [mc])
model = load_model('../models/densemodel2.h5')

y_pred = np.round(model.predict(X_test_transformed))
graph_model_history(history)
print(accuracy_score(y_test, y_pred))
plot_confusion(y_test, y_pred)



In [ ]:
def make_model3():
    model = Sequential()
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate = .1))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate = .1))    
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate = .1))

    model.add(Dense(1, activation='sigmoid'))


    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

    return model
mc = ModelCheckpoint(filepath='../models/densemodel3.h5', save_best_only=True, monitor='val_acc',
                    mode = 'max')
model = make_model3()
history = model.fit(X_train_transformed, y_train_transformed,
                  epochs = 100,
                  validation_data = (X_test_transformed,y_test),
                   batch_size = 500,
                   verbose = 0,
                   callbacks = [mc])
model = load_model('../models/densemodel3.h5')

y_pred = np.round(model.predict(X_test_transformed))
graph_model_history(history)
print(accuracy_score(y_test, y_pred))
plot_confusion(y_test, y_pred)


In [ ]:
def make_model4():
    model = Sequential()
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))

    model.add(Dense(1, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=1000,
        decay_rate=0.9)
    optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)    
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['acc'])

    return model
mc = ModelCheckpoint(filepath='../models/densemodel4.h5', save_best_only=True, monitor='val_acc',
                    mode = 'max')
model = make_model4()
history = model.fit(X_train_transformed, y_train_transformed,
                  epochs = 100,
                  validation_data = (X_test_transformed,y_test),
                  batch_size = 500,
                   verbose = 0,
                   callbacks = [mc])
model = load_model('../models/densemodel4.h5')

y_pred = np.round(model.predict(X_test_transformed))
graph_model_history(history)
print(accuracy_score(y_test, y_pred))
plot_confusion(y_test, y_pred)


In [ ]:
def make_model5():
    model = Sequential()
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(1, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=1000,
        decay_rate=.9)
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)    
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['acc'])

    return model
mc = ModelCheckpoint(filepath='../models/densemodel5.h5', save_best_only=True, monitor='val_acc',
                    mode = 'max')
model = make_model5()
history = model.fit(X_train_transformed, y_train_transformed,
                  epochs = 100,
                  validation_data = (X_test_transformed,y_test),
                    batch_size = 500,
                   verbose = 0,
                   callbacks = [mc])
model = load_model('../models/densemodel5.h5')

y_pred = np.round(model.predict(X_test_transformed))
graph_model_history(history)
print(accuracy_score(y_test, y_pred))
plot_confusion(y_test, y_pred)


In [ ]:
def make_model6():
    model = Sequential()
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(1, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=.9)
    optimizer = keras.optimizers.SGD(
        learning_rate=lr_schedule
    )    
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['acc'])

    return model
mc = ModelCheckpoint(filepath='../models/densemodel6.h5', save_best_only=True, monitor='val_acc',
                    mode = 'max')
model = make_model6()
history = model.fit(X_train_transformed, y_train_transformed,
                  epochs = 100,
                  validation_data = (X_test_transformed,y_test),
                  batch_size = 500,
                   verbose = 0,
                   callbacks = [mc])
model = load_model('../models/densemodel6.h5')

y_pred = np.round(model.predict(X_test_transformed))
graph_model_history(history)
print(accuracy_score(y_test, y_pred))
plot_confusion(y_test, y_pred)


In [ ]:
def make_model7():
    model = Sequential()
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(rate=.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(rate=.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(rate=.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(rate=.5))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.1))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate=.1))
    model.add(Dense(1, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=1000,
        decay_rate=.9)
    optimizer = keras.optimizers.Adam(
        learning_rate=lr_schedule
        )    
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['acc'])

    return model
mc = ModelCheckpoint(filepath='../models/densemodel7.h5', save_best_only=True, monitor='val_acc',
                    mode = 'max')
model = make_model7()
history = model.fit(X_train_transformed, y_train_transformed,
                  epochs =100,
                  validation_data = (X_test_transformed,y_test),
                  batch_size = 500,
                   verbose = 0,
                   callbacks = [mc])
model = load_model('../models/densemodel7.h5')

y_pred = np.round(model.predict(X_test_transformed))
graph_model_history(history)
print(accuracy_score(y_test, y_pred))
plot_confusion(y_test, y_pred)


In [ ]:
def make_model8():
    model = Sequential()
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(20, activation='relu'))
    model.add(Dropout(rate=.1))
    model.add(Dense(20, activation='relu'))
    model.add(Dropout(rate=.1))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate=.1))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=.7)
    optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)    
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['acc'])

    return model
mc = ModelCheckpoint(filepath='../models/densemodel8.h5', save_best_only=True, monitor='val_acc',
                    mode = 'max')
model = make_model8()
history = model.fit(X_train_transformed, y_train_transformed,
                    epochs = 100,
                    validation_data = (X_test_transformed,y_test),
                    batch_size = 500,
                   verbose = 0,
                   callbacks = [mc])
model = load_model('../models/densemodel8.h5')

y_pred = np.round(model.predict(X_test_transformed))
graph_model_history(history)
print(accuracy_score(y_test, y_pred))
plot_confusion(y_test, y_pred)


In [ ]:
for j in range(1,9):
    print(f'Model Number {j}')
    model = keras.models.load_model(f'../models/densemodel{j}.h5')
    y_pred = np.round(model.predict(X_test_transformed))
    eval_df = test_df
    eval_df['prediction'] = y_pred.astype(int)
    modules = eval_df['code_module'].unique()
    course_acc = pd.DataFrame(columns = ['score'], index = modules)
    fig, axes = plt.subplots(4,2, figsize = (10,15))
    axes = axes.ravel()
    for i, module in enumerate(modules):
        preds = eval_df[eval_df['code_module'] == module]['prediction']
        trues = eval_df[eval_df['code_module'] == module]['final_result']
        accuracy = accuracy_score(trues,preds)
        course_acc.loc[module,'score'] = accuracy
        plot_confusion(trues,preds, ax = axes[i])
        axes[i].set_title(f'Module: {module}: Accuracy Score: {round(accuracy,3)}')
        axes[i].set_yticklabels(['True Fail', 'True Pass'])
        axes[i].set_xticklabels(['Pred Fail','Pred Pass'])
    plot_confusion(y_test, y_pred, ax = axes[-1])
    accuracy = accuracy_score(y_test, y_pred)
    axes[-1].set_title(f'All Modules: Accuracy Score: {round(accuracy,3)}')
    axes[-1].set_yticklabels(['True Fail', 'True Pass'])
    axes[-1].set_xticklabels(['Pred Fail','Pred Pass'])
    plt.tight_layout(rect = [0,0,0,0])
    plt.suptitle(f'Module {j} Test Set Confusion Matrices by Module')
    plt.show()